# 🚀 Ultimate Temple Crowd Detection - YOLOv8 Pro Pipeline

**Features Included:**
- Uses Roboflow **Version 5** (Best & largest dataset)
- Trains **YOLOv8m** (Medium) for maximum accuracy on crowds
- **Advanced Fine-Tuning** (CosLR, Augmentations, Early Stopping)
- **Pro Evaluation** (Confusion Matrix, mAP Metrics)
- **Real-Life Testing** (Image & Video Inference)
- **Export & Download** (PyTorch `.pt` and Production `.onnx` formats)

**Instructions:**
Ensure you are running this in Google Colab with a **T4 GPU** enabled (`Runtime` > `Change runtime type` > `T4 GPU`).

In [ ]:
# 1. Setup Environment & Check GPU
!pip install -q ultralytics roboflow

import torch
from ultralytics import YOLO

print("✅ Setup Complete!")
print(f"🔥 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"💻 GPU Device: {torch.cuda.get_device_name(0)}")

In [ ]:
# 2. Download Best Dataset (Version 5) from Roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="wsdLXeUN3MmbPrvRMrqF") # Your API Key
project = rf.workspace("ricky-sambora").project("crowd-detection-7suou")

# Using Version 5 (The larger, better dataset)
version = project.version(5)
dataset = version.download("yolov8")

print(f"✅ Dataset Version 5 downloaded to: {dataset.location}")

In [ ]:
# ── Cell 3: Advanced Model Training & Fine-Tuning ──────────────────────────────
from ultralytics import YOLO
import os

# Initialize YOLOv8 Medium — best balance of speed and accuracy for crowd detection
model = YOLO('yolov8m.pt')

print("🚀 Starting advanced training pipeline on T4 GPU...")
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,           # Train up to 100 epochs (early stopping will cut short if needed)
    imgsz=640,            # Standard resolution — best balance for detection
    batch=16,             # Optimal for T4 (16GB VRAM); reduce to 8 if OOM error
    patience=25,          # Stop early if no improvement for 25 epochs
    device=0,             # Force GPU usage
    optimizer='auto',     # Ultralytics auto-selects AdamW or SGD
    cos_lr=True,          # Cosine LR schedule — smoother convergence than step decay
    lr0=0.01,             # Initial learning rate
    lrf=0.1,              # Final LR = lr0 * lrf = 0.001 (sensible decay)
    dropout=0.1,          # Light dropout to reduce overfitting
    plots=True,           # Auto-generate confusion matrix, F1 curve, PR curve, results.png
)

# ⚠️ IMPORTANT: Save the exact run directory — YOLO auto-increments (train, train2, train3...)
# Hardcoding 'runs/detect/train' WILL break if you run training more than once.
TRAIN_DIR = str(results.save_dir)
WEIGHTS_DIR = os.path.join(TRAIN_DIR, 'weights')

print(f"\n✅ Training Complete!")
print(f"📁 Train run saved to : {TRAIN_DIR}")
print(f"🏆 Best weights at   : {WEIGHTS_DIR}/best.pt")

In [ ]:
# ── Cell 4: Detailed Model Evaluation ─────────────────────────────────────────
from IPython.display import display, Image
from ultralytics import YOLO
import glob, os

# Load the BEST weights from this specific training run (uses dynamic path — never breaks)
best_pt = os.path.join(WEIGHTS_DIR, 'best.pt')
best_model = YOLO(best_pt)

print(f"✅ Loaded best weights: {best_pt}")
print(f"\n📋 Model summary:")
best_model.info()

# --- Validate on the validation split ---
metrics = best_model.val()

print("\n" + "=" * 50)
print("📊  FINAL VALIDATION METRICS")
print("=" * 50)
print(f"  mAP@50-95  (primary metric) : {metrics.box.map:.4f}")
print(f"  mAP@50                      : {metrics.box.map50:.4f}")
print(f"  mAP@75                      : {metrics.box.map75:.4f}")
print(f"  Precision                   : {metrics.box.mp:.4f}")
print(f"  Recall                      : {metrics.box.mr:.4f}")
print("=" * 50)
if metrics.box.map50 >= 0.80:
    print("🟢 Excellent — mAP@50 above 80%")
elif metrics.box.map50 >= 0.60:
    print("🟡 Good — mAP@50 above 60%")
else:
    print("🔴 Needs improvement — consider more data or epochs")

# --- Display training plots (all paths built dynamically from TRAIN_DIR) ---
plots_to_show = {
    'Training Loss & Validation Curves': 'results.png',
    'Confusion Matrix':                  'confusion_matrix.png',
    'F1-Confidence Curve':               'F1_curve.png',
    'Precision-Recall Curve':            'PR_curve.png',
}
for title, filename in plots_to_show.items():
    path = os.path.join(TRAIN_DIR, filename)
    if os.path.exists(path):
        print(f"\n📈 {title}:")
        display(Image(filename=path, width=800))
    else:
        print(f"⚠️  {filename} not found — skipping")

In [ ]:
# ── Cell 5: Real-Life Testing on Unseen Images ─────────────────────────────────
from IPython.display import display, Image
import glob, os

print("🔍 Testing best model on unseen test set images...")

# Try test/images first, fall back to valid/images if test split doesn't exist
test_images = glob.glob(f"{dataset.location}/test/images/*.jpg")
if not test_images:
    print("⚠️  No test/images found — falling back to valid/images for visual check")
    test_images = glob.glob(f"{dataset.location}/valid/images/*.jpg")

if not test_images:
    print("❌ No images found for inference. Check dataset download.")
else:
    # Use up to 5 test images
    sample_images = test_images[:5]
    print(f"📸 Running inference on {len(sample_images)} sample image(s)...\n")

    # Run prediction (saves annotated images to runs/detect/predictX/)
    predictions = best_model.predict(
        source=sample_images,
        save=True,           # saves annotated image to disk
        conf=0.25,           # 25% confidence threshold
        verbose=False
    )

    for r in predictions:
        # r.save_dir = the predict output directory (e.g. runs/detect/predict2)
        # r.path     = the source image path used for this prediction
        saved_img = os.path.join(r.save_dir, os.path.basename(r.path))
        if os.path.exists(saved_img):
            print(f"✅ {os.path.basename(r.path)} — {len(r.boxes)} detections")
            display(Image(filename=saved_img, width=500))
        else:
            # Fallback: glob the predict dir for any result image
            fallback = glob.glob(os.path.join(r.save_dir, '*.jpg'))
            if fallback:
                display(Image(filename=fallback[0], width=500))

    print("\n🎥 To test on a VIDEO file:")
    print("  1. Upload your .mp4 to Colab (left panel → Upload)")
    print("  2. Run: best_model.predict('your_video.mp4', save=True, conf=0.3)")
    print("  3. The annotated video will be saved in runs/detect/predictX/")

In [ ]:
# ── Cell 6: Export to Production Formats ──────────────────────────────────────
import os

print("📦 Exporting model for production use...")

# Export to ONNX — platform-independent, works in Python/Node.js without PyTorch
# dynamic=True allows variable batch sizes (required for live camera streams)
onnx_export_path = best_model.export(format='onnx', dynamic=True, simplify=True)

# Ultralytics returns the ONNX path — save it for the download step below
ONNX_PATH = str(onnx_export_path)

print(f"\n✅ Exports complete!")
print(f"  PyTorch weights : {os.path.join(WEIGHTS_DIR, 'best.pt')}")
print(f"  ONNX model      : {ONNX_PATH}")
print("\n📌 Use best.onnx in your Node.js/Python backend for fast inference without PyTorch overhead")

In [ ]:
# ── Cell 7: Download Trained Models to Your Windows PC ────────────────────────
# Each files.download() triggers a browser Save-As dialog (or saves to VS Code file explorer).
from google.colab import files
import os

# Paths built dynamically from TRAIN_DIR captured during training (never hardcoded)
PT_PATH   = os.path.join(WEIGHTS_DIR, 'best.pt')    # PyTorch weights
LAST_PATH = os.path.join(WEIGHTS_DIR, 'last.pt')    # Last epoch weights (backup)

download_targets = [
    (PT_PATH,   '🏆 best.pt   — best checkpoint (use this in production)'),
    (ONNX_PATH, '⚡ best.onnx — production ONNX (Node.js/Python, no PyTorch needed)'),
    (LAST_PATH, '💾 last.pt   — final epoch checkpoint (backup)'),
]

print("📥 Starting model downloads...\n")
for filepath, description in download_targets:
    if os.path.exists(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        files.download(filepath)
        print(f"  ✅ {description}")
        print(f"     Size: {size_mb:.1f} MB  →  {filepath}")
    else:
        print(f"  ❌ Not found (run previous cells first): {filepath}")

print("\n" + "=" * 55)
print("🎉 DONE! Next steps:")
print("=" * 55)
print("1. Copy best.pt   → ml-services/crowd-detection/models/best.pt")
print("2. Copy best.onnx → ml-services/crowd-detection/models/best.onnx")
print("3. Use best.onnx in your backend for live crowd detection")
print(f"\n📊 Training results folder: {TRAIN_DIR}")